# Exploring User Identification!

In [ ]:
import asyncio
import nest_asyncio  # For Jupyter Notebook


import json
import pickle

import numpy as np


from typing import Any, Dict, List, Literal, Optional, Type
from crewai.flow.flow import Flow, listen, start, router

from llm_foundation import logger
from llm_foundation.agent_types import Persona, Role

from crewai import Agent, Task, Crew, Process
from crewai.crews import CrewOutput
from crewai_tools import BaseTool
from hackathon.index import generate_embeddings, create_index, search_index, build_similar_entities
from hackathon.input_output_types import NamedEntities
from hackathon.retrieval_neo4j import chunk_ranker, retrieve_context, retrieve_similar_entities, pagerank
from hackathon.graph_neo4j import add_entities, add_relates_to_relationships, build_vector_index, add_similar_entities, clean_db, Neo4jClientFactory
from hackathon.tools import filter_named_entities, create_document_deduped_entities_dict, create_matrix_entity_ref_count
from hackathon.utils import build_document_structure, save_document_structure
from langchain.agents import load_tools
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from pydantic import BaseModel
from rich import print
from rich.pretty import pprint


from hackathon.graph_graphiti import GraphitiSearchTool

from dotenv import load_dotenv

load_dotenv()

# Required for Jupyter Notebook
nest_asyncio.apply()

# Nice Video
# https://www.youtube.com/watch?v=8PtGcNE01yo#:~:text=so%20now%20it's%20time%20for%20us%20to,are%20nothing%20more%20than%20a%20new%20decorator.&text=once%20we've%20randomly%20generated%20that%20value%20we're,start%20method%20finishes%20we're%20going%20to%20trigger.

In [ ]:
entity_master = Persona.from_yaml_file("PersonasCrewAI.yaml")

profiler_role: Role = entity_master.get_role("profiler")
graphiti_finder_role: Role = entity_master.get_role("graphiti_finder")

pprint(profiler_role)
pprint(graphiti_finder_role)

profiler: Agent = profiler_role.to_crewai_agent(verbose=True, allow_delegation=True,) #, tools=human_tools)
graphiti_finder: Agent = graphiti_finder_role.to_crewai_agent(verbose=True, allow_delegation=True, tools=[GraphitiSearchTool()])


In [ ]:
from hackathon.graph_neo4j import User

class UserIdentityOutput(BaseModel):
    name: Optional[str] = None
    last_name: Optional[str] = None    
    user_identified: bool = False
    question: Optional[str] = None



class UserIdentityValidationState(BaseModel):
    user_info: List[str] = []
    name: Optional[str] = None
    last_name: Optional[str] = None    
    user_identified: bool = False
    question: Optional[str] = None

    def user_context(self) -> str:
        return "\n".join(self.user_info)
    
    def get_extracted_user(self) -> User:
        assert self.user_identified and self.name is not None and self.last_name is not None, f"User not identified: {self}"
        return User(name=self.name, last_name=self.last_name)


test = UserIdentityValidationState()
test.model_dump()

In [ ]:
class UserIdentityFlow(Flow[UserIdentityValidationState]):
        
    @start()
    def entry_point(self):
        return "router"
    
    @router(entry_point)
    def router(self):
        if not self.state.user_identified:
            return "go_identify_user"
        else:
            return "go_validate_user"
        
    @listen("go_identify_user")
    def identify_user(self):
        identify_user = Task(
            description=profiler_role.tasks[0].description,
            expected_output=profiler_role.tasks[0].expected_output,
            agent=profiler,
            output_pydantic=UserIdentityOutput,    
        )

        profiler_crew = Crew(
            agents=[profiler],
            tasks=[identify_user],
            verbose=True,
            memory=True,
            process=Process.sequential,
            cache=False,
        )        
        user_context = self.state.user_context()
        logger.info(f"Identifying user! Context:\n{user_context}")
        crew_output: CrewOutput = profiler_crew.kickoff(inputs={"user_context": user_context})
        result: UserIdentityOutput = crew_output.pydantic
        self.state.name = result.name
        self.state.last_name = result.last_name
        self.state.user_identified = result.user_identified
        self.state.question = result.question
        if result.question is not None:
            self.state.user_info.append(result.question)
        
        # pprint(crew_output)

    @listen("go_validate_user")
    def validate_user(self):
        logger.info("Validating user in DB!!!!")
    
        logger.info(f"Identifying user! State:\n{self.state}")

        
        search_task = Task(
            description=graphiti_finder_role.tasks[0].description,
            expected_output=graphiti_finder_role.tasks[0].expected_output,
            agent=graphiti_finder,
        )

        # Create Crew with memory enabled
        crew = Crew(
            agents=[graphiti_finder],
            tasks=[search_task],
            process=Process.sequential,
            verbose=True,
        )

        logger.info("^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Calling Agents ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
        result: CrewOutput = crew.kickoff(inputs={"query": f"{self.state.name} {self.state.last_name}"})
        logger.info(".................................................................................")
        logger.info(f"Answer:\n{print(result.raw)}")
        logger.info(".................................................................................")        
            
            

# Identify User Crew

Generates a query if we don't have user name and surname

In [ ]:
flow = UserIdentityFlow()


In [ ]:

initial_state = UserIdentityValidationState(user_info=[], name="", last_name="")
logger.info("^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Calling Flow ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
flow.kickoff(inputs=initial_state.model_dump())
logger.info(".............................. Flow Result ......................................")
next_state = flow.state
logger.info(next_state)
logger.info(".................................................................................")


In [ ]:
next_state.user_info.append("Hey! I'm Francisco")
logger.info("^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Calling Flow ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
flow.kickoff(inputs=next_state.model_dump())
logger.info(".............................. Flow Result ......................................")
next_state = flow.state
logger.info(next_state)
logger.info(".................................................................................")


In [ ]:
next_state.user_info.append("Nah, why I should tell you?")
logger.info("^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Calling Flow ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
flow.kickoff(inputs=next_state.model_dump())
logger.info(".............................. Flow Result ......................................")
next_state = flow.state
logger.info(next_state)
logger.info(".................................................................................")


In [ ]:
next_state.user_info.append("OOOOOk, I'm Perez-Sorrosal")
logger.info("^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Calling Flow ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
flow.kickoff(inputs=next_state.model_dump())
logger.info(".............................. Flow Result ......................................")
next_state = flow.state
logger.info(next_state)
logger.info(".................................................................................")

In [ ]:
next_state

In [ ]:
logger.info("^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Calling Flow ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
flow.kickoff(inputs=next_state.model_dump())
logger.info(".............................. Flow Result ......................................")
next_state = flow.state
logger.info(next_state)
logger.info(".................................................................................")

In [ ]:

# MATCH (m:Entity)
# CALL db.index.fulltext.queryNodes('node_name_and_summary', "Fran ")
# YIELD node AS entity, score
# RETURN entity.name AS name, entity.name_embedding as embedding, score

            # def search_vector2(tx, query_embedding):
            #     cypher_query = f"""
            #     CALL db.index.vector.queryNodes('name_entity_index', 3, {query_embedding}) YIELD node, score
            #     RETURN node.node_id as id, node.name as name, score
            #     """
            #     return tx.run(cypher_query).data()


from hackathon.utils import Neo4jClientFactory
from hackathon.retrieval_neo4j import retrieve_similar_entities

identity_to_check = "Francisco Perez-Sorrosal" #f"{next_state.name} {next_state.surname}"


neo4j_factory = Neo4jClientFactory()

related_nodes = retrieve_similar_entities(neo4j_factory, [identity_to_check], emb_dim=256)

print(related_nodes)


for node in related_nodes:
    logger.info(node)


# User creation in DB

In [ ]:
# Fake state
next_state = UserIdentityValidationState(name="Fran", last_name="Perez-Sorrosal", user_identified=True, question="")

print(next_state.get_extracted_user())



In [ ]:
from hackathon.tools import graphdb_retrieval_tool, graphdb_add_user_tool

user_db_persona = Persona.from_yaml_file("UserDBCrewAI.yaml")

entity_retriever_role: Role = user_db_persona.get_role("entity_retriever")
user_manager_role: Role = user_db_persona.get_role("user_manager")

pprint(entity_retriever_role)
pprint(user_manager_role)


entity_retriever: Agent = entity_retriever_role.to_crewai_agent(verbose=True, allow_delegation=True, tools=[graphdb_retrieval_tool] ) #, tools=human_tools)
user_manager: Agent = user_manager_role.to_crewai_agent(verbose=True, allow_delegation=True, tools=[graphdb_add_user_tool])


retrieve_entity = Task(
    description=entity_retriever_role.tasks[0].description,
    expected_output=entity_retriever_role.tasks[0].expected_output,
    agent=entity_retriever,
)

add_user = Task(
    description=user_manager_role.tasks[0].description,
    expected_output=user_manager_role.tasks[0].expected_output,
    agent=user_manager,
)


manager = Agent(
    role="User Creator",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an database manager, skilled in overseeing complex task workflows to add users to a database and guiding your teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
)

profiler_crew = Crew(
    agents=[entity_retriever, user_manager],
    tasks=[retrieve_entity, add_user],
    manager_agent=manager,
    planning=True,
    verbose=True,
    memory=True,
    process=Process.sequential,
    cache=False,
)

user_inputs = {
    "entity_context": str(next_state.get_extracted_user()),  # Input the full name as a string
    "user_context": next_state.user_context(),  # Input the raw Pydantic object
}
crew_output: CrewOutput = profiler_crew.kickoff(inputs=user_inputs)

logger.info("^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Calling Flow ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
logger.info(crew_output.raw)
